<a href="https://colab.research.google.com/github/nanopiero/PREAC/blob/main/notebooks/Multimodal_Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Atelier 4 : Régression multimodale avec un Visual Transformer


In [1]:
# Imports des bibliothèques utiles
# pour l'IA
import torch
# pour les maths
import numpy as np
# pour afficher des images et des courbes
import matplotlib.pyplot as plt

In [2]:
! git clone https://github.com/nanopiero/PREAC.git

Cloning into 'PREAC'...
remote: Enumerating objects: 87, done.
remote: Counting objects: 100% (87/87), done.
remote: Compressing objects: 100% (84/84), done.
remote: Total 87 (delta 40), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (87/87), 7.17 MiB | 10.81 MiB/s, done.
Resolving deltas: 100% (40/40), done.


## A. Découverte du problème

In [3]:
pip install einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 1.8 MB/s eta 0:00:00


In [ ]:
from PREAC.utile_Transformers import voir_batch2D, gen_image_with_pairs, set_tensor_values

# Notre jeu de données contient:
# une cible parfaite (lamedeau)
# des triplets "pluviometres" :
# (lon_pluvio, lat_pluvio, taux de pluie mesuré)
# des quintuplets "cmls" associés aux antennes A & B:
# (lon_A, lat_A, lat_B, lon_B, taux de pluie moyen entre A et B)

batch_size = 6
n_pairs = 16
n_points = 16
lamedeau, pluviometres, radar, cmls_spatialises, cmls = gen_image_with_pairs(6, n_pairs, n_points)

# lame d'eau "idéale"
fig1 = plt.figure(1, figsize=(36, 6))
voir_batch2D(lamedeau, 6, fig1, k=0, min_scale=0, max_scale=1)

# images radar (bruitées)
fig2 = plt.figure(2, figsize=(36, 6))
voir_batch2D(radar, 6, fig2, k=0, min_scale=0, max_scale=1)

# Commercial Microwave Links (cmls)
fig3 = plt.figure(3, figsize=(36, 6))
voir_batch2D(cmls_spatialises, 6, fig3, k=0, min_scale=0, max_scale=1)

# Superposition Commercial Microwave Links (CMLs), pluviomètres et radar
fig4 = plt.figure(4, figsize=(36, 6))
cmls_spatialises = set_tensor_values(cmls_spatialises, pluviometres, 64)
radar[cmls_spatialises > 0] = cmls_spatialises[cmls_spatialises > 0 ]
voir_batch2D(radar, 6, fig4, k=0, min_scale=0., max_scale=1.2)


**Questions intéressantes** : \\
Pourquoi est-ce que le temps de génération des images est long la première fois qu'on lance le code, mais pas les suivantes ? \\
En quoi les cmls et les pluviomètres peuvent-ils aider à atteindre la cible (c'est à dire la lame d'eau complète) ? \\

## B. Traitement par FCN

A partir des ateliers précédents, il est possible de définir
une approche simple permettant de combiner les trois sources d'information.
Seule obstacle : comment concaténer les entrées. D'où le code suivant:

In [5]:
lamedeau, pluviometres, radar, cmls_spatialises, cmls = gen_image_with_pairs(6, n_pairs, n_points)

In [ ]:
lamedeau, pluviometres, radar, cmls_spatialises, cmls = gen_image_with_pairs(6, n_pairs, n_points)
pluviometres_spatialises =  -0.1 * torch.ones(radar.shape)
pluviometres_spatialises = set_tensor_values(pluviometres_spatialises, pluviometres, 64)
input = torch.cat([radar, pluviometres_spatialises, cmls_spatialises], dim = 1)
print(input.shape)

**Questions intéressantes** : \\
Pourquoi est-ce qu'on créé une matrice de -0.1 pour les pluviomètres spatialisés ? \\
Comment instancier un UNet pour prendre ce type d'input en entrée ? \\
Visualiser les sorties au bout de cinquante époques (100 batches de 32 par époque).

## C. Encodage des différentes variables qui vont alimenter le transformer

In [8]:
# Paramètres du modèle :
image_size = [64,64]
channels = 1
patch_size = 4
d_model = 120
mlp_expansion_ratio = 4
d_ff = mlp_expansion_ratio * d_model
n_heads = 4
n_layers = 12

In [9]:
# Module interne du réseau responsable de l'encodage des variables :
from PREAC.utile_Transformers import UnifiedEmbedding
ue = UnifiedEmbedding(d_model, patch_size, channels)


In [ ]:
lamedeau, pluviometres, radar, cmls_spatialises, cmls = gen_image_with_pairs(6, n_pairs, n_points)
embeddings = ue(radar, pluviometres, cmls)
print(embeddings.shape)

**Question intéressante** : \\
Comment interpréter les dimensions de l'input après encodage ? \\


## D. Entraînement du Transformer

In [ ]:
from PREAC.utile_Transformers import FusionTransformer
model = FusionTransformer(image_size, patch_size, n_layers, d_model, d_ff, n_heads, channels=1)
lamedeau, pluviometres, radar, cmls_spatialises, cmls = gen_image_with_pairs(6, n_pairs, n_points)
model(radar, pluviometres, cmls).shape

In [12]:
def criterion(output, target):
    return torch.abs((output - target)).mean()

import torch.optim as optim
optimizer = optim.Adam(model.parameters(), 10**(-4))

In [ ]:
nepochs = 50
nbatches = 100
batchsize = 32
train_losses = []
device = torch.device('cuda:0')
model = model.to(device)


for epoch in range(nepochs):
    print(f"Epoch {epoch + 1}/{nepochs}")

    epoch_losses = []

    for i in range(nbatches):

        ...

        epoch_losses.append(loss.detach().cpu().item())

    epoch_loss = np.mean(epoch_losses)
    train_losses.append(epoch_loss)

    print(f'Epoch loss: {epoch_loss:.4f}')


**Questions intéressantes** : \\
Quelle différence qualitative entre les outputs ? \\
Que doit faire le transformer "en plus", comparé au FCN ?



## E. Chargement d'un Transformer entraîné

In [16]:
# Avec France Transfert ??
# !curl 'https://francetransfert.numerique.gouv.fr/api-private/download-module/generate-download-url' -X POST \
# -H 'Content-Type: application/json' \
# -H 'Origin:https://francetransfert.numerique.gouv.fr' \
# --data-raw '{"enclosure":"164ea132-cf5e-4a8d-a084-62841b3122ec","recipient":"cGllcnJlLmxlcGV0aXRAbWV0ZW8uZnI%3D","token":"ddf68980-7b19-4eef-8a34-88a3e32a0f71","senderToken":null,"password":"2q*vbl62!FK@Z"}'

In [26]:
# Modèles entraînés sur 900 époques :
# mViT_900ep.pth comme au D.
# mViT_0radar_900ep.pth avec, au préalable: radar = 0 x radar
! wget https://www.grosfichiers.com/K3aaxZcSnX4_Fic8rPjJ9yZ
! unzip K3aaxZcSnX4_Fic8rPjJ9yZ
! rm K3aaxZcSnX4_Fic8rPjJ9yZ

--2024-04-25 22:47:25--  https://www.grosfichiers.com/K3aaxZcSnX4_Fic8rPjJ9yZ
Resolving www.grosfichiers.com (www.grosfichiers.com)... 51.68.254.173
Connecting to www.grosfichiers.com (www.grosfichiers.com)|51.68.254.173|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 52591268 (50M) [application/octet-stream]
Saving to: ‘K3aaxZcSnX4_Fic8rPjJ9yZ’

K3aaxZcSnX4_Fic8rPj 100%[===================>]  50.15M  8.91MB/s    in 7.5s    

2024-04-25 22:47:34 (6.71 MB/s) - ‘K3aaxZcSnX4_Fic8rPjJ9yZ’ saved [52591268/52591268]

Archive:  K3aaxZcSnX4_Fic8rPjJ9yZ
 extracting: mViT_0radar_900ep.pth   
 extracting: mViT_900ep.pth          


In [33]:
# charger un checkpoint avec torch .load
# visualiser les outputs

checkpoint = torch.load('mViT_900ep.pth')
model.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [ ]:
#visualization:

model.eval()

full_target, partial_target, noisy_images, traces, pairs_list = gen_image_with_pairs(6, n_pairs, n_points)
lamedeau, pluviometres, radar, cmls_spatialises, cmls = gen_image_with_pairs(6, n_pairs, n_points)


radar = radar.to(device)
cmls = cmls.to(device)
pluviometres = pluviometres.to(device)

outputs = model(radar, pluviometres, cmls)

radar = radar.cpu()
cmls = cmls.cpu()
pluviometres = pluviometres.cpu()
outputs = outputs.cpu().detach()

# lame d'eau "idéale"
fig1 = plt.figure(1, figsize=(36, 6))
voir_batch2D(lamedeau, 6, fig1, k=0, min_scale=0, max_scale=1)

# images radar (bruitées)
fig2 = plt.figure(2, figsize=(36, 6))
voir_batch2D(radar, 6, fig2, k=0, min_scale=0, max_scale=1)

# Commercial Microwave Links (cmls)
fig3 = plt.figure(3, figsize=(36, 6))
voir_batch2D(cmls_spatialises, 6, fig3, k=0, min_scale=0, max_scale=1)

# Superposition Commercial Microwave Links (CMLs), pluviomètres et radar
fig3 = plt.figure(3, figsize=(36, 6))
voir_batch2D(outputs, 6, fig3, k=0, min_scale=0, max_scale=1)



**Question intéressante** : \\
Le transformer parvient-il à exploiter les valeurs de pluviomètres et des cmls  ?

Bibliographie :  [Jaegle et al. 2020](https://arxiv.org/abs/1811.12739)